In [1]:
from libs.alibaba.alibaba import Alibaba
from libs.alibaba.inquiry import Inquiry
from libs.alibaba.inquiry import element_has_css_class
from libs.email import Email
from libs.json import JSON

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

from string import Template
from datetime import datetime
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import json
import requests
import time
import re
import math

import types
import traceback
import threading

import pendulum

import logging
from logging.handlers import TimedRotatingFileHandler

In [2]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
market

. storage markets.json
.\storage\markets.json was deserialized!


{'name': 'Eyelashes',
 'directory': 'E:\\glitter\\Eyelashes',
 'lid': 'eyelash@glittergroupcn.com',
 'lpwd': 'Wangzhen68970747',
 'homepage': 'https://glittereyelash.en.alibaba.com',
 'lname': 'Jeremy Wang',
 'accounts': [{'lid': 'eyelash1@glittergroupcn.com',
   'lpwd': 'Zhangying25160',
   'lname': 'Emily Zhang',
   'disabled': False},
  {'lid': 'eyelash2@glittergroupcn.com',
   'lpwd': 'Huyinping18130',
   'lname': 'Ada Alva',
   'disabled': False}]}

In [3]:
account = market['accounts'][1]
print(account)
inquiry = Inquiry(market, account, headless=False)

{'lid': 'eyelash2@glittergroupcn.com', 'lpwd': 'Huyinping18130', 'lname': 'Ada Alva', 'disabled': False}
E:\glitter\Eyelashes_config [] inquiry_tracking_ids_Ada.json
E:\glitter\Eyelashes_config\inquiry_tracking_ids_Ada.json was deserialized!


In [3]:
inquiry = Inquiry(market, headless=False)

E:\glitter\Tools_config [] inquiry_tracking_ids_Jessica.json
E:\glitter\Tools_config\inquiry_tracking_ids_Jessica.json was deserialized!
primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jessica


In [4]:
inquiry.login()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Ada


In [4]:
def webww_check(self):
    threads = []
    # try to login
    icon = WebDriverWait(self.browser, 15).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, '#webatm2-iconbar')))
    container = self.browser.find_element_by_css_selector('#webww-contacts')
    dialog_iframe = self.browser.find_element_by_css_selector('#webatm2-iframe')
    while True:
        icon.click()
        try:
            WebDriverWait(self.browser, 15).until(
                EC.invisibility_of_element_located((By.CSS_SELECTOR, '#webww-contacts .webatm2-tips')))
        except TimeoutException as e:
            print(str(e))
            traceback.print_exc()
        confirm = self.browser.find_element_by_css_selector('#webww-contacts .webatm2-confirm')
        if confirm.is_displayed():
            # failed login, cancel checking
            print('Trade Manager has been loged in somewhere else, cancel checking')
            close_btn = confirm.find_element_by_css_selector('span.webatm2-confirm-close')
            close_btn.click()
            return threads

        if 'show-panel' in container.get_attribute('class'):
            break

    panel = WebDriverWait(self.browser, 15).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, '#webatm2-panel')))
    # after login

    # open dialog
    time.sleep(2)
    tab_recent = panel.find_element_by_css_selector('div.tab-recent')
    if 'unread' not in tab_recent.get_attribute('class'):
        # tracking_tid_exists = False
        # for key in self.tracking_ids:
        #     if key.startswith('enali'):
        #         tracking_tid_exists = True
        #         break
        # if not tracking_tid_exists:
        #     print('there is no new messages.')
        #     return threads
        print('there is no new messages.')
        return threads

    self.load_tracking_ids()

    tab_sysmsg = panel.find_element_by_css_selector('div.tab-sysmsg')
    tab_sysmsg.click()
    panel.find_element_by_css_selector('.panel-sysmsg.active a').click()
    WebDriverWait(self.browser, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#webatm2-dialog')))

    # switch to iframe and navigate to recent threads
    self.browser.switch_to.frame(dialog_iframe)
    sidebar = self.browser.find_element_by_css_selector('.webatm2-sidebar')
    head = self.browser.find_element_by_css_selector('.webatm2-window-head')
    body = self.browser.find_element_by_css_selector('.webatm2-window-body')
    sidebar.find_element_by_css_selector('.tab-thread').click()
    time.sleep(1)
    doc = pq(sidebar.find_element_by_css_selector('.webatm2-thread-list').get_attribute('innerHTML'))
    for div in doc.find('.webatm2-thread'):
        div = pq(div)
        thread = {}
        tid = div.attr('data-user-id')
        thread['id'] = tid
        thread['online'] = not div.hasClass('offline')
        thread['name'] = div.find('.thread-info .name').text()
        thread['unread'] = div.hasClass('unread')
        thread['unread_count'] = int(div.find('.thread-info span.unread').text())
        threads.append(thread)

        if thread['unread_count'] == 0:
            continue

        if tid not in self.tracking_ids:
            self.tracking_ids[tid] = {}
            self.tracking_ids[tid]['datetime'] = pendulum.now()

            # 工具平台 不 区分 新老客户
            if 'tools' in self.market['name'].lower():
                self.tracking_ids[tid]['status'] = ['contacted']
            else:
                if re.search('[\u4E00-\u9FA5]+', thread['name']):  # contacted
                    self.tracking_ids[tid]['status'] = ['contacted']
                else:
                    self.tracking_ids[tid]['status'] = ['new']

            self.save_tracking_ids()
        self.webww_switch_to_thread(thread, sidebar, head, body)

    self.browser.switch_to.default_content()
    # self.find_('#webatm2-dialog .webatm2-dialog-close').click()

    # time.sleep(5)
    # self.load_url()
    return threads
inquiry.webww_check = types.MethodType(webww_check, inquiry)

In [ ]:
def webww_switch_to_thread(self, thread, sidebar, head, body):

        div = sidebar.find_element_by_css_selector('div.webatm2-thread[data-user-id="' + thread['id'] + '"]')

        if 'active' not in div.get_attribute('class'):
            # msgs = body.find_elements_by_css_selector('.webatm2-messages .webatm2-message')
            div.click()

        #     if msgs:
        #         print(msgs)
        #         WebDriverWait(self.browser, 15).until(EC.staleness_of(msgs[-1]))
        #     else:
        #         WebDriverWait(self.browser, 15).until(
        #             EC.presence_of_element_located((By.CSS_SELECTOR, '.webatm2-messages .webatm2-message')))
        #     time.sleep(1)
        # WebDriverWait(self.browser, 15).until(
        #     EC.text_to_be_present_in_element((By.CSS_SELECTOR, '.webatm2-window-head .user-name'), thread['name']))

        WebDriverWait(self.browser, 15).until(
            EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, '.show-contact iframe')))
        buyer_name = ''
        country = ''
        login_location = ''
        is_alibaba_employee = False

        try:
            buyer_name = self.browser.find_element_by_css_selector('#mini-card-wrap span.user-nick').text
            span_country = self.browser.find_element_by_css_selector(
                '#mini-card-wrap .user-base-info-wrap span:nth-child(2)')
            country = re.search(':(.*)', span_country.get_attribute('title')).group(1)

            spans = self.browser.find_elements_by_css_selector('#vaildEmployee')
            if spans:
                is_alibaba_employee = True

            p_login_location = self.browser.find_elements_by_css_selector('#mini-card-wrap p.last-login-time')
            if p_login_location:
                login_location = re.search(':(.*)', p_login_location[0].get_attribute('title')).group(1)
        except Exception as e:
            print(str(e))
            traceback.print_exc()
        finally:
            self.browser.switch_to.parent_frame()

        print(buyer_name, is_alibaba_employee, country, login_location)
        doc = pq(self.browser.find_element_by_css_selector('.webatm2-window-body .webatm2-messages').get_attribute(
            'innerHTML'))

        is_replied = False
        last_buyer_message = []
        messages = []
        for div in doc.find('.webatm2-message'):
            div = pq(div)
            message = {}
            message['is_self'] = div.hasClass('self')
            message['datetime'] = div.find('.time-tag').text() if div.find('.time-tag') else None
            message['content'] = div.find('.content').text().replace('\xa0', ' ')

            if message['is_self']:
                last_buyer_message = []
                is_replied = True
            else:
                last_buyer_message.append(message)
                is_replied = False

            messages.append(message)

        salutation = 'hello'
        if buyer_name:
            salutation = 'Dear ' + buyer_name

        # tempory code to set links for catalog
        link = ''
        if 'eyelash' in self.market['name'].lower():
            link = 'https://drive.google.com/file/d/1gfHwDl1qPomAMnkFGCjiVk74e8CQagoI'
        elif 'tool' in self.market['name'].lower():
            link = ''

        params = {'salutation': salutation, 'link': link, 'sender': self.lname, 'email': self.lid}

        tid = thread['id']
        tracking = self.tracking_ids[tid]
        last_status = tracking['status'][-1]
        
        if is_replied:
            pass
        elif is_alibaba_employee:
            msg = '0123456789'
            self.webww_send_message(msg, body)
            tracking['status'].append('irrelevant reply')
            self.save_tracking_ids()
        elif last_status == 'new':
            msg = self.reply_templates['webww_first_reply'].substitute(params)
            self.webww_send_message(msg, body)
            tracking['status'].append('contacted')
            self.save_tracking_ids()
        elif last_status == 'contacted':
            msg = self.reply_templates['webww_default_reply_for_contacted_buyer'].substitute(params)
            self.webww_send_message(msg, body)
            tracking['status'].append('replied with default message')
            self.save_tracking_ids()
        else:
            pass
inquiry.webww_switch_to_thread = types.MethodType(webww_switch_to_thread, inquiry)

In [ ]:
def webww_send_message(self, msg, body):
    edit = body.find_element_by_css_selector('div.webatm2-editor-body')
    for part in msg.split('\n'):
        edit.send_keys(part)
        print(part)
        ActionChains(self.browser).key_down(Keys.SHIFT).key_down(Keys.ENTER).key_up(Keys.SHIFT).perform()
    edit.send_keys(Keys.BACKSPACE)
    send_btn = body.find_element_by_css_selector('button.send')
#     send_btn.click()

inquiry.webww_send_message = types.MethodType(webww_send_message, inquiry)

In [4]:
inquiry.login()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Ada


In [ ]:
inquiry.browser.switch_to.default_content()

In [7]:
inquiry.load_url()

In [8]:
threads = inquiry.webww_check()

there is no new messages.


In [ ]:
dialog_iframe = inquiry.browser.find_element_by_css_selector('#webatm2-iframe')
inquiry.browser.switch_to.frame(dialog_iframe)
sidebar = inquiry.browser.find_ele.ment_by_css_selector('.webatm2-sidebar')
head = inquiry.browser.find_element_by_css_selector('.webatm2-window-head')
body = inquiry.browser.find_element_by_css_selector('.webatm2-window-body')


In [ ]:
thread = threads[0]
thread

In [ ]:
inquiry.webww_switch_to_thread(threads[1], sidebar, head, body)

In [ ]:
inquiry.browser.switch_to.default_content()

In [ ]:
for t in threads:
    inquiry.webww_switch_to_thread(t, sidebar, head, body)

In [ ]:
not {}

In [ ]:
inquiry.load_reply_templates()

In [ ]:
inquiry.reply_templates

In [ ]:
def get_inquiries(self):
    html = pq(self.browser.page_source)
    inquiry_list = html.find('.ui2-list-body .aui2-grid-wraper')
    inquiries = []
    for elem in inquiry_list:
        div = pq(elem)
        item = {}
        item['url'] = div.find('a').attr('href')
        text = div.find('div.spec-inquiry-id').text().strip()
        item['id'] = re.search('(\d+)', text).group(1)
        item['date'] = div.find('div.spec-inquiry-id+div').text().strip()
        item['tags'] = []
        for span in div.find('div.aui-tags span'):
            item['tags'].append(pq(span).text())
        item['is_replied'] = True if div.find('td:nth-child(2) i[title="已回复"]') else False
        item['title'] = div.find('td:nth-child(3)').text()
        item['buyer'] = div.find('td:nth-child(4)').text().strip()
        item['is_buyer_online'] = True if div.find('td:nth-child(5) div.atm-online') else False
        text = div.find('td:nth-child(6) i').attr('title')
        try:
            item['buyer_country'] = re.sub(' \[.*\]', '', text)
            item['buyer_local_time'] = re.search('\[Local Time: (.*)\]', text).group(1)
        except AttributeError:
            item['buyer_country'] = text
            item['buyer_local_time'] = None
        item['responsible_person'] = div.find('td:nth-child(7)').text()
        item['status'] = div.find('td:nth-child(8)').text()
        if '新询盘' in item['status'] or 'New Inquiry' in item['status']:
            item['is_new'] = True
        else:
            item['is_new'] = False
        inquiries.append(item)
    return inquiries

inquiry.get_inquiries = types.MethodType(get_inquiries, inquiry)

In [12]:
def send_message(self, message, attach=None):
    chat_form = self.browser.find_element_by_css_selector('form.reply-wrapper')
    chat_form.find_element_by_css_selector('div.holder').click()

    uploading_failed = False
    if attach:
        file_input = self.browser.find_element_by_css_selector('input#attachs')
        file_input.send_keys(attach)

        css = 'div.next-upload-list-item:last-child'
        div = WebDriverWait(self.browser, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css)))
        try:
            WebDriverWait(self.browser, 150).until(
                element_has_css_class(div, 'next-upload-list-item-done'))
        except TimeoutException:
            print('time out, cancel uploading')
            div.find_element_by_css_selector('i.next-icon-close').click()
            uploading_failed = True

    if uploading_failed:
        return False

    # if uploading_failed or attach is None:
    #     doc = pq(message)
    #     doc.find('span.attach').remove()
    #     message = doc.outer_html()

    WebDriverWait(self.browser, 15).until(
        EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, '#inquiry-content_ifr')))
    body = self.browser.find_element_by_tag_name('body')
    self.click(body)

    js = self.reply_js_template.format(message=message)
    self.browser.execute_script(js)
    self.browser.switch_to.default_content()

    time.sleep(1)

    btn_send = chat_form.find_element_by_css_selector('button.send')
    btn_send.click()
    
    return True

inquiry.send_message = types.MethodType(send_message, inquiry)

In [9]:

def reply(self, enquiry):
    try:
        self.open_inquiry_in_new_tab(enquiry)
        conversation = self.get_conversation()

        messages = conversation['messages']
        buyer = conversation['buyer']

        last_buyer_messages = []
        is_replied = False
        reply_count = 0
        for msg in messages:
            if msg['position'] == 'left':
                last_buyer_messages.append(msg)
                is_replied = False
            elif msg['position'] == 'right' and 'Seller Assistant' not in msg['user']:
                last_buyer_messages = []
                is_replied = True
                reply_count += 1

        # check if email address is already exists in conversation
        text = ''
        for msg in last_buyer_messages:
            text = text + '\n\n ' + msg['content']
        emails = re.findall('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', text)

        if buyer['email']:
            emails.append(buyer['email'])

        eid = enquiry['id']
        tracking = self.tracking_ids[eid]
        last_status = tracking['status'][-1]
        greetings = '<p>Pleased to hear from you.</p><br>'
        if last_status == 'new':
            if emails:
                mime_message = Email.message_of_product_catalog(self.market, self.account, buyer['name'])
#                 if Email.send(self.account, emails, mime_message):
                if True:
                    params = {'buyer': buyer['name'], 'greetings': greetings, 'email': ','.join(emails), 'sender': self.lname}
                    message = self.reply_templates['notify_catalog_was_sent'].substitute(params)
                    if self.send_message(message, self.catalog):
                        tracking['status'].append('catalog was sent by email')
                        self.save_tracking_ids()
                    else:
                        tracking['status'].append('catalog was sent by email, but reply was failed')
                        self.save_tracking_ids()
            else:
                params = {'buyer': buyer['name'], 'greetings': greetings, 'sender': self.lname}
                message = self.reply_templates['notify_first_reply_without_email'].substitute(params)
                if self.send_message(message, self.catalog):
                    tracking['status'].append('catalog was sent directly')
                    self.save_tracking_ids()
        elif 'catalog was sent' in last_status and 'reply was failed' in last_status:
            if is_replied:
                tracking['status'].append('catalog was sent by email')
                self.save_tracking_ids()
            else:
                params = {'buyer': buyer['name'], 'greetings': greetings, 'email': ','.join(emails), 'sender': self.lname}
                message = self.reply_templates['notify_catalog_was_sent'].substitute(params)
                if self.send_message(message, self.catalog):
                    tracking['status'].append('catalog was sent by email')
                    self.save_tracking_ids()

        elif 'catalog was sent' in last_status:
            pass
        elif 'wait for email address' in last_status:
            text = ''
            for msg in last_buyer_messages:
                text = text + '\n\n ' + msg['content']
            emails = re.findall('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', text)
            print(emails)
            if emails:
                tracking['emails'] = emails
                self.save_tracking_ids()
                mime_message = Email.message_of_product_catalog(self.market, self.account, buyer['name'])
                if Email.send(self.account, emails, mime_message):
                    params = {'buyer': buyer['name'], 'greetings': '', 'email': ', '.join(emails),
                              'sender': self.lname}
                    message = self.reply_templates['notify_catalog_was_sent'].substitute(params)
                    if self.send_message(message, self.catalog):
                        tracking['status'].append('catalog was sent by email')
                        self.save_tracking_ids()
    except Exception as e:
        self.notify("danger", '回复 询盘 [' + enquiry['id'] + '] 时 发生错误! ' + str(e))
        traceback.print_exc()
    finally:
        self.close_inquiry_and_switch_back()
inquiry.reply = types.MethodType(reply, inquiry)

In [ ]:
inquiry.load_url()
time.sleep(0.1)

In [ ]:
inquiries = inquiry.get_inquiries()

In [ ]:
enquiry = inquiries[3]
enquiry

In [ ]:
inquiry.is_auto_reply_needed(enquiry)

In [ ]:
inquiry.reply(enquiry)

In [ ]:
inquiry.account

In [ ]:
inquiry.load_reply_templates()
inquiry.reply_templates['notify_first_reply_without_email']

In [ ]:
inquiry.tracking_ids['11947313067']['status'] = ['new']
inquiry.tracking_ids

In [ ]:
inquiry.check()

In [13]:
inquiry.check()

E:\glitter\Eyelashes_config [] inquiry_tracking_ids_Ada.json
E:\glitter\Eyelashes_config\inquiry_tracking_ids_Ada.json was deserialized!
==------------------------------------------------------==
ada alva ada alva
enquiry 100732618307 reply is needed
danger 回复 询盘 [100732618307] 时 发生错误! 'NoneType' object has no attribute 'click'


Traceback (most recent call last):
  File "<ipython-input-9-4661a9914e6c>", line 42, in reply
    if self.send_message(message, self.catalog):
  File "<ipython-input-12-8c643104bcf2>", line 44, in send_message
    btn_send.click()
AttributeError: 'NoneType' object has no attribute 'click'


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.17134 x86_64)


In [ ]:
inquiry.tracking_ids

In [ ]:
inquiry.close_inquiry_and_switch_back()

In [ ]:
inquiry.load_find_catalog()

In [ ]:
inquiry.catalog

In [ ]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
inquiry = Inquiry(market)

In [ ]:
emails = ['changshu.qd@gmail.com', 'odingdongo@hotmail.com']
mime_message = Email.message_of_product_catalog(inquiry.market, inquiry.account, 'prestl bernhard')
Email.send(inquiry.account, emails, mime_message)

In [ ]:
inquiry.account

In [ ]:
enquiry = inquiries[8]
enquiry

In [ ]:
enquiry = inquiries[8]
inquiry.open_inquiry_in_new_tab(enquiry)
conversation = inquiry.get_conversation()
inquiry.close_inquiry_and_switch_back()
messages = conversation['messages']
buyer = conversation['buyer']
messages

In [ ]:
enquiry = inquiries[8]
enquiry

In [ ]:
# inquiry.load_tracking_ids()
inquiry.tracking_ids

In [ ]:
inquiry.close_inquiry_and_switch_back()

In [ ]:
# inquiry.close_inquiry_and_switch_back()
# inquiry.tracking_ids['11947313067']['status'] = ['new']
enquiry = inquiries[0]
inquiry.reply(enquiry)

In [ ]:
inquiry.tracking_ids

In [ ]:
def try_login_webww():
    btn_login = WebDriverWait(inquiry.browser, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#webatm2-iconbar')))
    btn_login.click()

In [ ]:
panel = inquiry.browser.find_element_by_css_selector('#webatm2-panel')

In [ ]:
inquiries[11]

In [ ]:
re.findall('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', text)

In [ ]:
inquiry.browser.save_screenshot("screenshot.png")

In [ ]:
text = 'Inquiry ID：100708962003'

In [ ]:
text = '询价单号：100708962003'

In [ ]:
re.search('(\d+)', text).group(1)

In [ ]:
text

In [ ]:
msg = 'ldskfjsl'
js = "document.querySelector('.webatm2-window-body .webatm2-editor-body').innerHTML = `{message}`;"
js.format(message=msg)

In [ ]:
from string import Template

In [ ]:
params = {'salutation': 'chang', 'link': 'link', 'sender': 'shu'}
templ = Template('$salutation, lsdkfjlskf. $sender')
msg = templ.substitute(params)

In [ ]:
msg